In [49]:
import pandas as pd
import csv


# Import dataset
data = []
min = 999
max = 0

file_path = 'bac_date-deschise_2023-ses1.csv'
 
# Use on_bad_lines to skip lines with too many fields
bac_data = pd.read_csv(file_path, on_bad_lines='skip')
 
# Display the DataFrame
bac_data = bac_data[bac_data['STATUS'] == 'Promovat']
columns_to_drop = ['Forma de învățământ', 'Unitate (SIRUES)', 'Clasa', 'Subiect ea', 'Subiect eb','Limba modernă','Subiect ec','Subiect ed','Promoție',
                   'NOTE_RECUN_A','NOTE_RECUN_B','NOTE_RECUN_C','NOTE_RECUN_D','NOTE_RECUN_EA','NOTE_RECUN_EB','NOTE_RECUN_EC','NOTE_RECUN_ED','STATUS_A',
                   'STATUS_B','STATUS_C','STATUS_D','STATUS_EA','STATUS_EB','STATUS_EC','STATUS_ED','ITA','SCRIS_ITC','SCRIS_PMS','ORAL_PMO','ORAL_IO',
                     'CONTESTATIE_EA','NOTA_CONTESTATIE_EA','CONTESTATIE_EB','NOTA_CONTESTATIE_EB','CONTESTATIE_EC','NOTA_CONTESTATIE_EC','CONTESTATIE_ED',
                     'NOTA_CONTESTATIE_ED','PUNCTAJ DIGITALE']
bac_data = bac_data.drop(columns=columns_to_drop)
# print(bac_data.head(20))


file_path = '425220906-coduri-sirues-xls.csv'
school_data = pd.read_csv(file_path, on_bad_lines='skip')
school_data['Cod SIRUES'] = school_data['Cod SIRUES'].head(10).astype(int)
print(school_data.head(10))
# Filter data



   Nr. Judeţ                                           Denumire  Cod SIRUES  \
0    1  Alba  LICEUL TEHNOLOGIC AGRICOL "ALEXANDRU BORZA" CI...    844547.0   
1    2  Alba                          SCOALA GIMNAZIALA SOHODOL    307408.0   
2    3  Alba            SCOALA GIMNAZIALA "I. M. MOLDOVAN" BLAJ    303983.0   
3    4  Alba         SCOALA GIMNAZIALA "PETRU PAVEL AARON" BLAJ    304004.0   
4    5  Alba                COLEGIUL TEHNIC "APULUM" ALBA IULIA   1062421.0   
5    6  Alba         SCOALA GIMNAZIALA "AVRAM IANCU" ALBA IULIA    303660.0   
6    7  Alba            COLEGIUL NATIONAL "TITU MAIORESCU" AIUD   3107263.0   
7    8  Alba          LICEUL TEHNOLOGIC "STEFAN MANCIULEA" BLAJ    844559.0   
8    9  Alba                            LICEUL TEHNOLOGIC SEBES    100654.0   
9   10  Alba        SCOALA GIMNAZIALA "LUCIAN BLAGA" OCNA MURES    304236.0   

             Fel unitate               Tip unitate  \
0  Unitate de învăţământ       Grup şcolar agricol   
1  Unitate de învăţămâ